In [1]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import pandas as pd
import numpy as np
import utility_functions2 as uf
import string
import itertools
import timeit
import scipy.spatial.distance as dist


In [18]:
%%memit
n, m = 3000, 3000
name_gen = map(''.join, itertools.islice(itertools.product('ABCDE',repeat=6), n))
attr_gen = map(''.join, itertools.islice(itertools.product('EDCBA',repeat=6), m))
names = list(name_gen)
attr = list(attr_gen)

peak memory: 241.20 MiB, increment: 0.00 MiB


In [19]:
%%memit
# data = np.triu(np.random.random((n, m)))
# data = (data + data.T) / 2
data = np.random.randint(2, size=(n, m))
data

peak memory: 371.48 MiB, increment: 130.28 MiB


In [20]:
#%%memit
df = pd.DataFrame(data, index=names, columns=attr)
df2 = df.astype('boolean')
#df

In [21]:
%%time
uf.createSimilarityMatrix(df, 'jaccard')

CPU times: user 28.1 s, sys: 234 ms, total: 28.3 s
Wall time: 28.3 s


,AAAAAA,AAAAAB,AAAAAC,AAAAAD,AAAAAE,AAAABA,AAAABB,AAAABC,AAAABD,AAAABE,...,AEDEDA,AEDEDB,AEDEDC,AEDEDD,AEDEDE,AEDEEA,AEDEEB,AEDEEC,AEDEED,AEDEEE
AAAAAA,1.000000,0.339938,0.342731,0.321476,0.350133,0.357332,0.339875,0.347885,0.328609,0.339974,...,0.354497,0.345011,0.338496,0.344980,0.335258,0.336391,0.349355,0.330512,0.334521,0.332300
AAAAAB,0.339938,1.000000,0.345559,0.319556,0.352993,0.342370,0.334371,0.338104,0.326106,0.344498,...,0.354967,0.347265,0.337760,0.338326,0.338085,0.335081,0.348620,0.335123,0.329631,0.330381
AAAAAC,0.342731,0.345559,1.000000,0.322365,0.347443,0.334635,0.332447,0.333185,0.336004,0.356705,...,0.351209,0.335827,0.324147,0.339349,0.334368,0.351435,0.331870,0.322566,0.336603,0.343904
AAAAAD,0.321476,0.319556,0.322365,1.000000,0.333333,0.333482,0.296460,0.329540,0.322187,0.342819,...,0.331261,0.329290,0.331078,0.339503,0.322959,0.328438,0.324082,0.323489,0.309140,0.320576
AAAAAE,0.350133,0.352993,0.347443,0.333333,1.000000,0.354881,0.342246,0.339429,0.327449,0.343439,...,0.349693,0.336703,0.346803,0.337867,0.337029,0.335220,0.343943,0.338847,0.350112,0.344781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AEDEEA,0.336391,0.335081,0.351435,0.328438,0.335220,0.348187,0.330834,0.340463,0.336142,0.349110,...,0.354258,0.341259,0.345438,0.340650,0.356084,1.000000,0.344371,0.327434,0.351752,0.330396
AEDEEB,0.349355,0.348620,0.331870,0.324082,0.343943,0.352941,0.345029,0.333184,0.333036,0.333911,...,0.347137,0.337024,0.327016,0.340567,0.331998,0.344371,1.000000,0.340397,0.336620,0.342152
AEDEEC,0.330512,0.335123,0.322566,0.323489,0.338847,0.320771,0.328372,0.307795,0.323582,0.343515,...,0.331418,0.339036,0.342689,0.333035,0.342441,0.327434,0.340397,1.000000,0.322377,0.310222
AEDEED,0.334521,0.329631,0.336603,0.309140,0.350112,0.320087,0.336047,0.342909,0.331828,0.345117,...,0.350313,0.335871,0.328718,0.337657,0.330783,0.351752,0.336620,0.322377,1.000000,0.315977


In [12]:
%%timeit
uf.createSimilarityMatrix(df2, 'jaccard')

142 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%time
uf.createSimilarityMatrix(df.astype(np.bool), 'jaccard')

CPU times: user 13.4 s, sys: 234 ms, total: 13.6 s
Wall time: 13.6 s


,AAAAAA,AAAAAB,AAAAAC,AAAAAD,AAAAAE,AAAABA,AAAABB,AAAABC,AAAABD,AAAABE,...,AEDEDA,AEDEDB,AEDEDC,AEDEDD,AEDEDE,AEDEEA,AEDEEB,AEDEEC,AEDEED,AEDEEE
AAAAAA,1.000000,0.339938,0.342731,0.321476,0.350133,0.357332,0.339875,0.347885,0.328609,0.339974,...,0.354497,0.345011,0.338496,0.344980,0.335258,0.336391,0.349355,0.330512,0.334521,0.332300
AAAAAB,0.339938,1.000000,0.345559,0.319556,0.352993,0.342370,0.334371,0.338104,0.326106,0.344498,...,0.354967,0.347265,0.337760,0.338326,0.338085,0.335081,0.348620,0.335123,0.329631,0.330381
AAAAAC,0.342731,0.345559,1.000000,0.322365,0.347443,0.334635,0.332447,0.333185,0.336004,0.356705,...,0.351209,0.335827,0.324147,0.339349,0.334368,0.351435,0.331870,0.322566,0.336603,0.343904
AAAAAD,0.321476,0.319556,0.322365,1.000000,0.333333,0.333482,0.296460,0.329540,0.322187,0.342819,...,0.331261,0.329290,0.331078,0.339503,0.322959,0.328438,0.324082,0.323489,0.309140,0.320576
AAAAAE,0.350133,0.352993,0.347443,0.333333,1.000000,0.354881,0.342246,0.339429,0.327449,0.343439,...,0.349693,0.336703,0.346803,0.337867,0.337029,0.335220,0.343943,0.338847,0.350112,0.344781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AEDEEA,0.336391,0.335081,0.351435,0.328438,0.335220,0.348187,0.330834,0.340463,0.336142,0.349110,...,0.354258,0.341259,0.345438,0.340650,0.356084,1.000000,0.344371,0.327434,0.351752,0.330396
AEDEEB,0.349355,0.348620,0.331870,0.324082,0.343943,0.352941,0.345029,0.333184,0.333036,0.333911,...,0.347137,0.337024,0.327016,0.340567,0.331998,0.344371,1.000000,0.340397,0.336620,0.342152
AEDEEC,0.330512,0.335123,0.322566,0.323489,0.338847,0.320771,0.328372,0.307795,0.323582,0.343515,...,0.331418,0.339036,0.342689,0.333035,0.342441,0.327434,0.340397,1.000000,0.322377,0.310222
AEDEED,0.334521,0.329631,0.336603,0.309140,0.350112,0.320087,0.336047,0.342909,0.331828,0.345117,...,0.350313,0.335871,0.328718,0.337657,0.330783,0.351752,0.336620,0.322377,1.000000,0.315977


In [5]:
%mprun -f uf.createSimilarityMatrix uf.createSimilarityMatrix(df.T, 'cosine')

                   '''
   216                                 Creates a similarity matrix between the rows of the inputDF based on
   217                                 the metric specified. The resulting matrix has both rows and columns labeled
   218                                 by the index of inputDF.
   219                                 '''
   220    197.4 MiB     34.3 MiB       similarity_matrix = dist.pdist(inputDF, metric)
   221    231.7 MiB     34.4 MiB       similarity_matrix = dist.squareform(similarity_matrix)
   222    231.7 MiB      0.0 MiB       similarity_matrix = 1 - similarity_matrix
   223    231.7 MiB      0.0 MiB       similarity_df = pd.DataFrame(
   224    231.7 MiB      0.0 MiB           data=similarity_matrix, index=inputDF.index, columns=inputDF.index)
   225    231.7 MiB      0.0 MiB       similarity_df.index.name = None
   226    231.7 MiB      0.0 MiB       similarity_df.columns.name = None
   227    231.7 MiB      0.0 MiB       return(similarity_df)

In [6]:
#%time uf.saveData(df, 'Output/', 'similarity_triu_savez_compressed', compression='npz', symmetric=False, dtype=np.float32)

In [8]:
#%%time
#df2 = uf.loadData('Output/similarity_triu_savez_compressed_2020_06.npz')
#df2

In [9]:
#assert(np.allclose(df, df2))

In [10]:
#%time uf.saveData(df, 'Output/', 'similarity_tsv', compression='gzip')